# Initial NBA-API testing

In [43]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
import pandas as pd

# 1) Find a player ID (e.g., Nikola Jokic)
all_players = players.get_players()
jokic = [p for p in all_players if p["full_name"] == "Nikola Jokić"][0]
print("Found player:", jokic)

# 2) Get game logs for this season (2024-25 season format: '2024-25')
gamelog = playergamelog.PlayerGameLog(
    player_id=jokic["id"],
    season='2025-26',
    season_type_all_star='Regular Season'
)

df = gamelog.get_data_frames()[0]
print(df.head())



Found player: {'id': 203999, 'full_name': 'Nikola Jokić', 'first_name': 'Nikola', 'last_name': 'Jokić', 'is_active': True}
  SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0     22025     203999  0022500248  Nov 17, 2025  DEN vs. CHI  L   39   13   
1     22025     203999  0022500232  Nov 15, 2025    DEN @ MIN  W   37    8   
2     22025     203999  0022500224  Nov 12, 2025    DEN @ LAC  W   34   18   
3     22025     203999  0022500212  Nov 11, 2025    DEN @ SAC  W   33   16   
4     22025     203999  0022500189  Nov 08, 2025  DEN vs. IND  W   32   10   

   FGA  FG_PCT  ...  DREB  REB  AST  STL  BLK  TOV  PF  PTS  PLUS_MINUS  \
0   27   0.481  ...    14   18   13    1    2    2   2   36          10   
1   15   0.533  ...     9   12   11    2    1    6   4   27           3   
2   23   0.783  ...    10   12    6    1    1    2   1   55          28   
3   19   0.842  ...    10   15    7    2    0    3   3   35          11   
4   14   0.714  ...     9   14   

In [ ]:
from nba_helpers_tier1 import evaluate_pra_bet

result = evaluate_pra_bet(
    player_name="Alex Sarr",
    line=15,          # PRA line you see at the book
    odds=-250,          # odds for OVER
    season="2024-25",
    season_type="Regular Season",
    home=False,          # only home games
    last_n=20,          # last 15 home games
    min_minutes=25,     # only games with 30+ min
    method="blend",     # blend empirical + normal
)

from pprint import pprint
pprint(result)


{'analytics': {'edge_prob_points': 0.07615062464236566,
               'ev_per_unit_staked': 0.10661087449931195,
               'fair_odds_over': -377},
 'empirical': {'line': 15,
               'max_pra': np.int64(45),
               'mean_pra': np.float64(22.3),
               'min_pra': np.int64(10),
               'n_games': 20,
               'p_over': 0.8,
               'p_push': 0.05,
               'p_under': 0.15,
               'std_pra': np.float64(8.772564889292191)},
 'filters': {'home': False, 'last_n': 20, 'min_minutes': 25},
 'model': {'method': 'blend', 'p_model_over': 0.79043633892808},
 'normal': {'line': 15,
            'mean_pra': np.float64(22.3),
            'n_games': 20,
            'p_over_normal': 0.7808726778561599,
            'std_pra': np.float64(8.772564889292191)},
 'player_name': 'Alex Sarr',
 'pra_line': 15,
 'season': '2024-25',
 'season_type': 'Regular Season',
 'sportsbook': {'implied_prob_over': 0.7142857142857143},
 'sportsbook_odds_over': -250

In [2]:
# Cell 2: import the module (make sure nba_tier2_model.py is in the same directory)
from pprint import pprint

from nba_helpers_tier2 import evaluate_tier2_stat_bet


In [10]:
# Cell 4: evaluate PRA over bet

result_pra = evaluate_tier2_stat_bet(
    player_name="Nikola Jokić",
    stat_type="pra",
    line=45,               # sportsbook PRA line
    odds=-115,               # American odds for OVER
    expected_minutes=34.0,   # your minutes projection for tonight
    seasons=["2022-23", "2023-24", "2024-25", "2025-26"],
    season_type="Regular Season",
    half_life_games=20.0,    # recent games weigh more
    min_minutes_for_fit=30,
    min_games_for_fit=15,
    direction="over",
)

pprint(result_pra)


{'analytics': {'edge_prob_points': 0.1534952680031888,
               'ev_per_unit': 0.2869694140929181,
               'fair_odds': -221},
 'direction': 'over',
 'distribution': {'mu': 50.48063353860682, 'sigma': 10.138471941692695},
 'expected_minutes': 34.0,
 'line': 45,
 'model_params': {'alpha': 10.835726370519245,
                  'beta': 1.1660266814143403,
                  'n_games_used': 206,
                  'sigma': 10.138471941692695},
 'odds': -115,
 'player_name': 'Nikola Jokić',
 'probabilities': {'p_book_win_implied': 0.5348837209302325,
                   'p_model_win': 0.6883789889334213},
 'season_type': 'Regular Season',
 'seasons_used': ['2022-23', '2023-24', '2024-25', '2025-26'],
 'stat_column': 'PRA',
 'stat_type': 'pra'}


In [21]:
# Cell 5: evaluate multiple props for same game

player_name = "Russell Westbrook"
seasons = ["2022-23", "2023-24", "2024-25", "2025-26"]
expected_minutes = 28

props = [
    # {"stat_type": "points",   "line": 26.5, "odds": -110, "direction": "over"},
    # {"stat_type": "rebounds", "line": 8, "odds": -260, "direction": "over"},
    # {"stat_type": "assists",  "line": 8.5,  "odds": -115, "direction": "over"},
    {"stat_type": "threes",   "line": 2,  "odds": -160, "direction": "over"},
    {"stat_type": "pra",      "line": 25, "odds": -200, "direction": "over"},
]

results = []
for prop in props:
    res = evaluate_tier2_stat_bet(
        player_name=player_name,
        stat_type=prop["stat_type"],
        line=prop["line"],
        odds=prop["odds"],
        expected_minutes=expected_minutes,
        seasons=seasons,
        season_type="Regular Season",
        half_life_games=20,
        min_minutes_for_fit=30,
        min_games_for_fit=20,
        direction=prop["direction"],
    )
    results.append(res)

for r in results:
    print("\n==========", r["stat_type"].upper(), "==========")
    print("Line:", r["line"], r["direction"], "@", r["odds"])
    print("E[stat] at", r["expected_minutes"], "min:", round(r["distribution"]["mu"], 2),
          "±", round(r["distribution"]["sigma"], 2))
    print("Model win prob:", round(r["probabilities"]["p_model_win"], 3))
    print("Book implied:",  round(r["probabilities"]["p_book_win_implied"], 3))
    print("Edge (pct pts):", round(r["analytics"]["edge_prob_points"] * 100, 1))
    print("Fair odds:", r["analytics"]["fair_odds"])
    print("EV per unit:", round(r["analytics"]["ev_per_unit"], 3))



========== THREES ==========
Line: 2 over @ -160
E[stat] at 28 min: 1.55 ± 1.5
Model win prob: 0.263
Book implied: 0.615
Edge (pct pts): -35.2
Fair odds: 280
EV per unit: -0.573

========== PRA ==========
Line: 25 over @ -200
E[stat] at 28 min: 26.25 ± 7.42
Model win prob: 0.54
Book implied: 0.667
Edge (pct pts): -12.6
Fair odds: -117
EV per unit: -0.19
